In [ ]:
import os
import time

import pandas as pd
import numpy as np

import cartopy
import fiona

from xml.dom import minidom
import zipfile

import geopandas as gpd

from shapely import geometry
from shapely.ops import nearest_points

import geoviews as gv
import geoviews.feature as gf
from geoviews import opts, tile_sources as gvts

from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.tile_providers import Vendors, get_provider
from bokeh.layouts import gridplot
from bokeh.models import Band, ColumnDataSource
from bokeh.models import Label, LabelSet, Range1d
from bokeh.palettes import Plasma4, Plasma6, Plasma10
from bokeh.layouts import column, gridplot

import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

output_notebook()

# SHANNON - **S**oil c**H**aracterization c**AN**ada basi**N** p**O**lygo**N**s

18 Feb. 2021 - Dan Kovacek

Python library requirements are above, most are straightforward and can be installed by your package manager (`pip` or `conda`), however you may have to install pygeos to your system and restart the kernel at the polygon intersection step.  This notebook was created and tested on Ubuntu 18 only, so YMMV!

## Purpose 

Retrieve data from [Gridded Soil Landscapes of Canada](https://sis.agr.gc.ca/nsdb/ca/cac003/cac003.20110308.v3.2/ca_all_slc_v3r2.zip) (GSLC), and use basin polygons to characterize soil information specific to the watersheds of interest.

## Gridded Soil Landscapes of Canada (GSLC) Data

While the [specifications](https://www.agr.gc.ca/atlas/supportdocument_documentdesupport/soilLandscapesOfCanada90mGrid/en/ISO_19131_Gridded_SLC_Data_Product_Specification.pdf) of the GSLC describe the data as 'gridded', it looks as though each row of the **GSLC** data (`data/ca_all_slc_v3r2/`) defines a polygon associated with a unique set of characteristics.  The `POLY_ID` column is the unique identifier for an area with a defined set of soil characteristics.  The large number of files makes it difficult to grasp the information contents of the GSLC in its entirety, but it seems that the POLY_ID is the common (or primary) key connecting the different information sources.  Further information about how the GSLC data is formatted is [here](https://sis.agr.gc.ca/cansis/nsdb/slc/v3.2/change.html), i.e. '*SLC map polygons are described by a set of soil components, defined by soil codes that are unique to each province.*'  Soil names and soil layers can be found [here](https://sis.agr.gc.ca/cansis/nsdb/soil/v2/download.html).  A visual mapping of the SLC data model is [here](https://sis.agr.gc.ca/cansis/nsdb/slc/v3.2/model.html).

## How others have summarized the information

A presentation of slides describing development of the SLC is [here](https://www.nrcs.usda.gov/Internet/FSE_DOCUMENTS/nrcs142p2_051155.pdf).  Slide 3 shows what the polygons actually look like.  Slide 7 of [this presentation](https://www.nrcs.usda.gov/Internet/FSE_DOCUMENTS/nrcs142p2_052032.pdf) talks about the SLC **component table.**

## Spatial Information

A tricky step in combining disparate data sources is managing coordinate reference systems (CRS).  Different data sources often use different coordinate reference systems, and the GSLC information comes in many different file types that are handled by different Python libraries.  Spatial data uses some coordinate reference system (CRS) and there are [many, many, many](https://spatialreference.org/) of these because the earth ~~is flat~~ is not perfectly spherical, so the deviations from a sphere cause inaccuracy if you try using one projection over too large an area.    

In general, for plotting shapes and points on top of some kind of base mapping, we need to use a geographic [web mercator CRS](https://www.esri.com/news/arcuser/0312/national-geographic-basemap.html).  In Google maps, as well as many base maps used in Python libraries, the base map coordinates are expressed in decimal degrees (EPSG 4326), so any geometries must be converted to the *geometric CRS* of the base map for plotting.  The CRS can be converted to a *projected CRS* for other operations, such as calculating distance or area (this isn't strictly necessary, but you need to deal with geometric decimal conversions if you don't).

Python libraries use an EPSG code (EPSG is the public registry of geodetic datums) to convert between datums/projections.  The SLC polygons are in the *Canada Albers Equal Area Conic* projection of the NAD83 datum.  The [EPSG for the SLC is 4269](https://epsg.io/4269).  For overlaying geographic information on 'web mercator' tiles, the [EPSG code used here is 4326](https://epsg.io/4326) (the common WGS84 used in GPS).  For spatial calculations, use NAD83 / BC Albers (EPSG 3005).

## Method

1. Import the WSC watershed basin polygons
    * download the National hydrometric network basin polygons, and save the zip file under `data/`. The zip archive doesn't need to be extracted.
2. Import custom polygons made in Google Earth and exported 
    * save these under `data/google_earth_polygons`
3. Verify coordinate systems and merge the polygons of interest into one geodataframe.
4. Calculate catchment parameters of interest
    * area and perimeter (Gravelius compactness can then be calculated as the basin perimeter divided by the perimeter of a circle with area equal to the basin area:
        $$K_G = \frac{P}{2\sqrt{\pi A}}$$
4. Import GSLC polygons.
5. Perform an intersection of the basin polygons with the soil polygons.
6. Use the output of the intersection to determine the basin composition by polygon ID.
7. Cross reference polygon ID against other data sources to find soil characteristics of interest.

## Import WSC Watershed Polygons

In [ ]:
# my file is located elsewhere, so delete the line below
WSC_basin_polygons_path = os.path.abspath(os.path.join(os.getcwd(), '../../hydat_db/'))
# if you followed the instructions in the introduction above, 
# you should have saved the file under data/, so delete the line above
# and use the one below instead
# WSC_basin_polygons_path = 'data/'

In [ ]:
# get all wsc_catchment data into its own dataframe
gdb_path = os.path.join(WSC_basin_polygons_path, 'WSC_Basins.gdb.zip')
all_layers = fiona.listlayers(gdb_path)
all_layer_names = [e.split('_')[1].split('_')[0] for e in all_layers]

# replace this with some call to define the sites of interest
all_sites = ['08MH147', '08ME002'] # 08MH147 is Stave River, 08ME002 is Lillooet River
filtered_layers = list(set(all_sites).intersection(all_layer_names))

In [ ]:
def get_polygon(stn):
    """
    Retrieve a watershed basin polygon based on the WSC Station ID.
    """
    gdb_path = os.path.join(WSC_basin_polygons_path, 'WSC_Basins.gdb.zip')
    data = gpd.read_file(gdb_path, driver='FileGDB', layer='EC_{}_1'.format(stn))
    # view the original CRS
    print(data.crs)
    # convert to WGS 84 / Pseudo-Mercator -- Spherical Mercator, Google Maps, OpenStreetMap, Bing, ArcGIS, ESRI
    # NOTE: for distance calculations, revert to original CRS (EPSG 4269) to get correct values
    data = data.to_crs(4269)
    return data

In [ ]:
shape_array = [get_polygon(site) for site in all_sites]

In [ ]:
# put the basin polygons in a geopandas geodataframe
all_basin_polygons = gpd.GeoDataFrame(pd.concat(shape_array, ignore_index=True))
all_basin_polygons
all_basin_polygons.crs

## Import custom polygons from Google Earth

In [ ]:
# import a test polygon created in Google Earth
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

def kmz_to_kml(fname):
    """
    Save kmz to kml.
    From: https://community.esri.com/t5/python-snippets-documents/convert-kmz-to-kml/ta-p/914947
    """
    zf = zipfile.ZipFile(fname, 'r')
    for fn in zf.namelist():
        if fn.endswith('.kml'):
            content = zf.read(fn)
            xmldoc = minidom.parseString(content)
            out_name = (fname.replace(".kmz",".kml")).replace("\\","/")
            out = open(out_name,'w')
            out.writelines(xmldoc.toxml())
            out.close()
        else:
            print("no kml file")


In [ ]:
custom_polygon_fp = 'data/google_earth_polygons/'
# custom_files = [e for e in os.listdir(custom_polygon_fp) if e.split('.')[-1] in ['kmz', 'kml']]
kmz_files = [e for e in os.listdir(custom_polygon_fp) if e.split('.')[-1] == 'kmz']
for k in kmz_files:
    kmz_to_kml(custom_polygon_fp + k)


## Merge the WSC basin polygons and custom GE-derived polygons into one GeoDataFrame

For geometric calculations, the [BC Environment Standard Projection is BC Albers Equal Area Conic](https://ibis.geog.ubc.ca/~brian/Course.Notes/bceprojection.html) (EPSG:3005).  See also [spatialreference.org](https://spatialreference.org/ref/epsg/3005/).

In [ ]:
def import_custom_polygons(custom_files):
    i = len(all_basin_polygons)
    for custom_file in custom_files:
        name = custom_file.split('.')[0].split('_')[-1]
        custom_shape = gpd.read_file(f'data/google_earth_polygons/{custom_file}', driver='kml')
        print(f'Original CRS of {name} polygon is {custom_shape.crs}')
        print(custom_shape.geometry)
        # kml files look like they are in 4326 by default (decimal degrees), 
        # though this could just be the Google Earth settings.

        # 3005
        custom_shape = custom_shape.to_crs(3005)
        custom_area = custom_shape['geometry'].area[0]/ 1E6
        custom_length = custom_shape['geometry'].length[0]/ 1E3
        print(f'Updated CRS of {name} polygon is {custom_shape.crs}')
        print(f'{name} basin area is {custom_area:.1f} km^2')
        print(f'{name} basin perimeter is {custom_length:.1f} km^2')
        custom_shape = custom_shape.to_crs(4269)
        print(f'Updated CRS of {name} polygon is {custom_shape.crs}')
        print('')
        # add our custom google earth kml
        print(all_basin_polygons.crs)
        all_basin_polygons.loc[i, 'Station'] = name
        all_basin_polygons.loc[i, 'StationNam'] = name
        all_basin_polygons.loc[i, 'Stn_UID'] = None
        all_basin_polygons.loc[i, 'Shp_Area'] = custom_area
        all_basin_polygons.loc[i, 'Shp_Perime'] = custom_length
        all_basin_polygons.loc[i, 'Shape_Length'] = custom_length * 10**3
        all_basin_polygons.loc[i, 'Shape_Area'] = custom_area * 10**6
        all_basin_polygons.loc[i, 'geometry'] = custom_shape.geometry[0]
        i += 1
    return custom_shape

In [ ]:
custom_kml_files = [e for e in os.listdir(custom_polygon_fp) if e.split('.')[-1] == 'kml']
custom_shapes = import_custom_polygons(custom_kml_files)

In google earth, the Snowshoe Creek basin area is 307 $km^2$ and the perimeter is $76.7 km^2$.   ( < 1% difference in area calculations).




In [ ]:
all_basin_polygons

## Calculate Basin Characteristics and Convert to plotting CRS

In [ ]:
basins_df = gpd.GeoDataFrame(all_basin_polygons['geometry'], crs='EPSG:4269')
basins_df = basins_df.to_crs(3005)
basins_df['station'] = all_basin_polygons['Station']
basins_df['gravelius'] = all_basin_polygons['Shp_Perime'] / (2 * np.sqrt(np.pi * all_basin_polygons['Shp_Area']))
basins_df['centroid'] = all_basin_polygons['geometry'].centroid
print(basins_df.crs)
print(basins_df.total_bounds)

# convert back to 4326 for plotting against geo tiles
basins_df = basins_df.to_crs(epsg=4269)
print(basins_df.crs)
print(basins_df.total_bounds)

basins_df

## Plot basins, basin centroids, and map Gravelius coeff. to colour

In [ ]:
# convert back to 4326 for plotting against geo tiles
basin_centroids = gpd.GeoDataFrame({'geometry': basins_df['centroid'].copy()})
basin_centroids = basin_centroids.to_crs(4326)

In [ ]:
centroids = gv.Points(basin_centroids)

In [ ]:
gv.Polygons(basins_df, vdims=['gravelius']).opts(colorbar=True, 
                                                 alpha=0.5,
                                                 frame_height=400,
                                                 color_index='gravelius',
                                                 clabel='Gravelius',
                                                data_aspect=True) * gvts.EsriNatGeo * centroids.opts(size=10, color='red')

## Import the GSLC Data

[Download the GSLC dataset](https://www.agr.gc.ca/atlas/data_donnees/geo/soilLandscapesOfCanada90mGrid/tif/gridded_slc_90m.zip), and extract the zip archive in the `data/` folder, or change the `shape_path` string below to match the file path.

In [ ]:
# Gridded SLC data product specification says EPSG: 3978 - NAD1983
# but at import the data suggests 4269

shape_path = 'data/ca_all_slc_v3r2/ca_all_slc_v3r2.shp'

with fiona.open(shape_path, "r") as shapefile:
    # note this is consistent with the data specification
    print(shapefile.crs)
    
    areas = [s['properties']['AREA'] for s in shapefile]
    perims = [s['properties']['PERIMETER'] for s in shapefile]
    poly_IDs = [s['properties']['POLY_ID'] for s in shapefile]
    eco_IDs = [s['properties']['ECO_ID'] for s in shapefile]

    
    geometries = [geometry.Polygon(s['geometry']['coordinates'][0]) for s in shapefile]

In [ ]:
# first format the imported data into a regular pandas dataframe
geo_dict = {'POLY_ID': poly_IDs,
           'ECO_ID': eco_IDs,
            'area': areas,
            'perimeter': perims,
           'geometry': geometries}
geo_df = pd.DataFrame(geo_dict)

In [ ]:
geo_df

In [ ]:
# now convert the data to a geopandas geodataframe
# and set the CRS for plotting
gdf = gpd.GeoDataFrame(geo_df, crs='epsg:4269')
print(gdf.crs)

print(gdf.total_bounds)
# gdf.plot()

In [ ]:
# compare the SL dataset and watershed dataset total bounds
basins_df = basins_df.to_crs(4269)
print(basins_df.total_bounds)
print(f'basins_df crs = {basins_df.crs}')



# gdf = gdf.to_crs(4269)
print(gdf.total_bounds)
print(f'SLC crs = {gdf.crs}')

## Find the Soil Lanscape Polygon Intersections with Basins

The `gpd.overlay` function may give some issues.  The error I was getting suggested I needed rtree (which was installed) or pygeos (which made shapely stop working).  Some info [here](https://geopandas.org/install.html).  I uninstalled and reinstalled geopandas, then reinstalled pygeos, performed some ancient rite of mysticism and it just worked.

The overlay function worked incredibly quickly to process the entire GSLC polygon set to find the overlapping zones.  

## So satisfying.

In [ ]:
res_intersection = gpd.overlay(gdf, basins_df, how='intersection')

In [ ]:
res_intersection

**Note that the area, perimeter, Gravelius, and centroid values are incorrect** because the data is in the geometric decimal degree format.  The dataframe crs has to be changed back to a mercator projection to get units in metres.

In the printout below of the resulting table, the basins are split by the soil information polygons, so a pandas `groupby` operation will let you iterate over all the sub-polygons of each basin to get fractional areas, etc.

On the figure below, you can hover to get the POLY_ID.  Neat.

In [ ]:
gv.Polygons(res_intersection, vdims=['POLY_ID']).opts(tools=['hover'],
                                                cmap='Spectral',
                                                alpha=0.8,
                                                frame_width=250,
                                                data_aspect=True) * gvts.EsriNatGeo

In [ ]:
res_intersection['idx'] = res_intersection.index
res_intersection.sort_values('POLY_ID', inplace=True)
res_intersection = res_intersection.to_crs(3005)
res_intersection['area'] = res_intersection['geometry'].area / 1E6
res_intersection['perimeter'] = res_intersection['geometry'].length / 1E3
res_intersection

## WIP: address the issue with the polygon areas not being correct

In [ ]:
intersection_table = pd.DataFrame(res_intersection)

In [ ]:
intersection_table

## TODO: Incorporate additional soil information files

* Harvey's area in GE is $6.7 km^2$
* Magnesia's area in GE is $4.9 km^2$

The differences between GE polygon areas and those calculated below are not constant multipliers.  They range between $1.5^2 \rightarrow 2^2$.

In [ ]:
intersection_table.groupby('station').sum()['area']